データを学習させるノートブック（予測値とモデル出力まで）

# 岡本編集

## 必要なライブラリインポート

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import cohen_kappa_score, make_scorer
import pandas as pd
import pickle
import numpy as np

## データ読み込み

In [ ]:
train_df = pd.read_csv('./data/processed_train.csv')
train_df.head()

In [ ]:
test_df = pd.read_csv('./data/processed_test.csv')
test_df.head()

## データ分割

In [ ]:
def hold_out(df,columns):
    x = df.drop(columns=columns)
    y = df.score
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = hold_out(train_df, ['essay_id', 'full_text', 'score'])

## RandomForestClassifier + GridSearchCV

### 評価指標作成

In [ ]:
kappa_score = make_scorer(cohen_kappa_score)

### 探索空間定義・学習

In [ ]:
param_grid = {'n_estimators': [100,300,500,1000], 'max_depth': [1,3,5,7,10], 'random_state': [42]}

In [ ]:
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring=kappa_score)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print(f'train_set_score: {grid_search.score(X_train,y_train)}')
print(f'test_set_score: {grid_search.score(X_test,y_test)}')
print(f'best_params: {grid_search.best_params_}')
print(f'best_cross_validation: {grid_search.best_score_}')

### 精度確認

In [ ]:
train_pred = grid_search.predict(X_train)
test_pred = grid_search.predict(X_test)

In [ ]:
#訓練データの精度
print(f'訓練データ精度：{cohen_kappa_score(y_train, train_pred)}')
#テストデータ精度
print(f'テストデータ精度：{cohen_kappa_score(y_test, test_pred)}')

### 予測値出力

In [ ]:
pred = grid_search.predict(test_df.drop(columns=['essay_id','full_text']))

In [ ]:
submit = pd.concat([test_df.essay_id,pd.DataFrame(pred,columns=['score'])],axis=1)
submit

In [ ]:
submit.to_csv('submission.csv',index=False)

### モデル出力

In [ ]:
pickle.dump(grid_search,open('./model/RandomForestClassifier_1.pkl','wb'))

## RandomForest(GridSearchのパラメーターを使用)

In [ ]:
clf = RandomForestClassifier(random_state=42,max_depth=5,n_estimators=500)

### 学習

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

### 予測精度

In [ ]:
#訓練データの精度
print(f'訓練データ精度：{cohen_kappa_score(y_train, train_pred)}')
#テストデータ精度
print(f'テストデータ精度：{cohen_kappa_score(y_test, test_pred)}')

### モデル出力

In [ ]:
pickle.dump(clf,open('./model/RandomForestClassifier_2.pkl','wb'))

### 予測値出力

In [ ]:
pred = clf.predict(train_df.drop(columns=['essay_id','full_text','score']))

In [ ]:
#予測値を訓練データに結合
processed_train_df = pd.concat([train_df,pd.DataFrame(pred,columns=['pred'])],axis=1)
processed_train_df

In [ ]:
processed_train_df.isnull().sum()

### 予測値格納後データフレーム格納

In [ ]:
#予測値を結合した訓練データを"processed_df_by_rf"としてcsv出力
processed_train_df.to_csv('./data/processed_df_by_rf.csv',index=False)

## LogisticRegressor + GridSearchCV